# Lab 8: SQL

## Course Policies

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** at the top
of your solution.


## Due Date

This assignment is due at **11:59pm Sunday, October 27th**.

# Collaborators

Write names in this cell:

In [1]:
import pandas as pd
import numpy as np
import re
import sqlalchemy

from dsua_112_utils import fetch_and_cache
from pathlib import Path

## Objectives for Lab 8:

We are going to practice viewing, sorting, grouping, and merging tables with SQL. 

# SQL

In the rest of the lab, we explore the money donated during the 2016 election using the [Federal Election
Commission's public records](http://www.fec.gov/finance/disclosure/ftpdet.shtml). You will be connecting to a sqlite database containing the data. The data we will be working with in this lab is quite small (16MB); however, it's taken from a larger database that more than few GBs!

### Connecting to SQLite

Download the database to local disk drive; this will take about a minute.

In [2]:
db_url = 'http://bit.ly/ds-ua-112-lab-08-db'
db_filename = 'fec_nyc.db'
data_dir = Path('data')
fetch_and_cache(db_url, db_filename, data_dir=data_dir)
db_path = data_dir / db_filename

Downloading... Done!


The following cell will connect to the above database using the SQL Alchemy library. This library is more general than the `sqlite3` module from the standard library because it can connect to several different database management systems, such as MySQL and PostgreSQL. It also supports an advanced feature for generating queries called an [object relational mapper](https://docs.sqlalchemy.org/en/latest/orm/tutorial.html) or ORM, which we won't discuss in this course but is quite useful for application development.

In [3]:
engine = sqlalchemy.create_engine(f"sqlite:///{db_path}")
connection = engine.connect()

In [4]:
engine.dispose()

To check that everything is working we will list the available tables.

In [5]:
engine.table_names()

['cand', 'comm', 'indiv_sample_nyc']

## Table Descriptions

Here is a list of the tables in the database.  Each table links to the documentation on the [FEC page](http://www.fec.gov/finance/disclosure/ftpdet.shtml) for the dataset.

Note that the table names here are slightly different from the ones in lecture. Consult the FEC page
for the descriptions of the tables to find out what the correspondence is.

- [`cand`](http://www.fec.gov/finance/disclosure/metadata/DataDictionaryCandidateMaster.shtml): Candidates table. Contains names and party affiliation.
- [`comm`](http://www.fec.gov/finance/disclosure/metadata/DataDictionaryCommitteeMaster.shtml): Committees table. Contains committee names and types.
- [`indiv_sample_nyc`](http://www.fec.gov/finance/disclosure/metadata/DataDictionaryContributionsbyIndividuals.shtml): All individual contributions from New York City.

We can explore the schemas of these tables with the following code from lecture 8.

In [6]:
for exp in connection.execute('SELECT sql FROM sqlite_master;'):
    print(exp[0])

CREATE TABLE "cand" (
    cand_id character varying(9),
    cand_name text,
    cand_pty_affiliation character varying(3),
    cand_election_yr integer,
    cand_office_st character varying(2),
    cand_office character(1),
    cand_office_district integer,
    cand_ici character(1),
    cand_status character(1),
    cand_pcc character varying(9),
    cand_st1 text,
    cand_st2 text,
    cand_city text,
    cand_st character varying(2),
    cand_zip character varying(10)
)
CREATE TABLE "comm"(
  "cmte_id" TEXT,
  "cmte_nm" TEXT,
  "tres_nm" TEXT,
  "cmte_st1" TEXT,
  "cmte_st2" TEXT,
  "cmte_city" TEXT,
  "cmte_st" TEXT,
  "cmte_zip" TEXT,
  "cmte_dsgn" TEXT,
  "cmte_tp" TEXT,
  "cmte_pty_affiliation" TEXT,
  "cmte_filing_freq" TEXT,
  "org_tp" TEXT,
  "connected_org_nm" TEXT,
  "cand_id" TEXT
)
CREATE TABLE indiv_sample_nyc (
    cmte_id character varying(9),
    amndt_ind character(1),
    rpt_tp character varying(3),
    transaction_pgi character(5),
    image_num bigint,
    trans

In [7]:
# We use `LIMIT 5` to avoid loading a huge table.
# Although our tables shouldn't get too large to display,
# this is generally good practice when working in the
# notebook environment.  Jupyter notebooks don't handle
# very large outputs well. 
query = """
SELECT * from indiv_sample_nyc LIMIT 5
"""
pd.read_sql(query, engine)

,cmte_id,amndt_ind,rpt_tp,transaction_pgi,image_num,transaction_tp,entity_tp,name,city,state,...,employer,occupation,transaction_dt,transaction_amt,other_id,tran_id,file_num,memo_cd,memo_text,sub_id
0,C00445015,N,Q1,P,15951128130,15,IND,"SINGER, TRIPP MR.",NEW YORK,NY,...,"ATLANTIC MAILBOXES, INC.",OWNER,01302015,1000,,A-CF13736,1002485,,,4041420151241812398
1,C00510461,N,Q1,P,15951129284,15E,IND,"SIMON, DANIEL A",NEW YORK,NY,...,N/A,RETIRED,03292015,400,C00401224,VN8JBDDJBA8,1002590,,* EARMARKED CONTRIBUTION: SEE BELOW,4041420151241813640
2,C00422410,N,Q1,P,15970352211,15,IND,"ABDUL RAUF, FEISAL",NEW YORK,NY,...,THE CORDOBA INITIATIVE,CHAIRMAN,03042015,250,,VN8A3DBSYG6,1003643,,,4041620151241914560
3,C00510461,N,Q1,P,15951129280,15,IND,"SCHWARZER, FRANK",NEW YORK,NY,...,METRO HYDRAULIC JACK CO,SALES,01162015,100,,VN8JBDAP4C4,1002590,,* EARMARKED CONTRIBUTION: SEE BELOW,4041420151241813630
4,C00510461,N,Q1,P,15951129281,15,IND,"SCHWARZER, FRANK",NEW YORK,NY,...,METRO HYDRAULIC JACK CO,SALES,02162015,100,,VN8JBDBRDG3,1002590,,* EARMARKED CONTRIBUTION: SEE BELOW,4041420151241813632


In [8]:
query = '''
SELECT cand_id, cand_name
FROM cand
WHERE cand_pty_affiliation = 'DEM'
LIMIT 5
'''
pd.read_sql(query,engine)

,cand_id,cand_name
0,H0AL05049,"CRAMER, ROBERT E ""BUD"" JR"
1,H0AL07086,"SEWELL, TERRYCINA ANDREA"
2,H0AL07094,"HILLIARD, EARL FREDERICK JR"
3,H0AR01091,"GREGORY, JAMES CHRISTOPHER"
4,H0AR01109,"CAUSEY, CHAD"


### Notes on Grading 

1. For the rest of the lab, we will be grading your SQL expressions so be sure to do all computation in SQL (unless otherwise requested).  
1. Use the query string variable we provide since we will be analyzing your SQL query strings.

---
## Question 3

For this question we will use regular expressions and sampling to extract data out of the `indiv_sample_nyc` table.

In [9]:
for exp in connection.execute('SELECT sql FROM sqlite_master WHERE name="indiv_sample_nyc";'):
    print(exp[0])

CREATE TABLE indiv_sample_nyc (
    cmte_id character varying(9),
    amndt_ind character(1),
    rpt_tp character varying(3),
    transaction_pgi character(5),
    image_num bigint,
    transaction_tp character varying(3),
    entity_tp character varying(3),
    name text,
    city text,
    state character(2),
    zip_code character varying(12),
    employer text,
    occupation text,
    transaction_dt character varying(9),
    transaction_amt integer,
    other_id text,
    tran_id text,
    file_num bigint,
    memo_cd text,
    memo_text text,
    sub_id bigint
)


### Question 3a

Let's first select transactions in the `indiv_sample_nyc` table with a `cmte_id` (committee id) that ends in 5. Then, groupby the transactions in the `indiv_sample_nyc` table with the `cmte_id` column, and count how many transactions and how much total amount of contribution there are for each `cmte_id`. Select the top 5 committees ordered by the transaction count.

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>committee_id</th>
      <th>total_amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>C00575795</td>
      <td>25099091</td>
      <td>110657</td>
    </tr>
    <tr>
      <th>1</th>
      <td>C00000935</td>
      <td>6989835</td>
      <td>36237</td>
    </tr>
    <tr>
      <th>2</th>
      <td>C00035675</td>
      <td>316019</td>
      <td>3417</td>
    </tr>
    <tr>
      <th>3</th>
      <td>C00040535</td>
      <td>492567</td>
      <td>3277</td>
    </tr>
    <tr>
      <th>4</th>
      <td>C00012245</td>
      <td>499659</td>
      <td>2128</td>
    </tr>
  </tbody>
</table>

```
BEGIN QUESTION
name: q3a
```

In [10]:
# BEGIN SOLUTION
query_q3a = '''
SELECT
    cmte_id AS committee_id,
    sum(transaction_amt) AS total_amount,
    count(*) AS count
FROM indiv_sample_nyc
WHERE cmte_id LIKE '%5'
GROUP BY cmte_id
ORDER BY count DESC
LIMIT 5
'''
# END SOLUTION

res_q3a = pd.read_sql(query_q3a, engine)
res_q3a

,committee_id,total_amount,count
0,C00575795,25099091,110657
1,C00000935,6989835,36237
2,C00035675,316019,3417
3,C00040535,492567,3277
4,C00012245,499659,2128


In [11]:
# TEST
res_q3a.shape
# EXPECTED: (5, 3)

(5, 3)

In [12]:
# TEST
all(res_q3a == res_q3a.sort_values('count', ascending=False))
# EXPECTED: True

True

In [13]:
# TEST
print(sorted(res_q3a['total_amount'].unique()))
# EXPECTED: [316019, 492567, 499659, 6989835, 25099091]

[316019, 492567, 499659, 6989835, 25099091]


### Question 3b

When you printed the schema for the tables, you may have noticed that both the `cand` and `comm` tables have a `cand_id` column. Let's try joining these two tables on this column.

List the first 5 candidate names (`cand_name`) in reverse lexicographic order by `cand_name`, along with their corresponding committee names. Only select rows that have a matching `cand_id` in both tables.

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cand_name</th>
      <th>cmte_nm</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ZUTLER, DANIEL PAUL MR</td>
      <td>CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT</td>
    </tr>
    <tr>
      <th>1</th>
      <td>ZUMWALT, JAMES</td>
      <td>ZUMWALT FOR CONGRESS</td>
    </tr>
    <tr>
      <th>2</th>
      <td>ZUKOWSKI, ANDREW GEORGE</td>
      <td>ZUKOWSKI FOR CONGRESS</td>
    </tr>
    <tr>
      <th>3</th>
      <td>ZUCCOLO, JOE</td>
      <td>JOE ZUCCOLO FOR CONGRESS</td>
    </tr>
    <tr>
      <th>4</th>
      <td>ZORN, ROBERT ERWIN</td>
      <td>CONSTITUTIONAL COMMITTEE</td>
    </tr>
  </tbody>
</table>

```
BEGIN QUESTION
name: q3b
```

In [22]:
# BEGIN SOLUTION
query_q3b = '''
SELECT cand.cand_name, comm.cmte_nm
FROM cand INNER JOIN comm ON cand.cand_id = comm.cand_id
ORDER BY cand_name DESC
LIMIT 5
'''
# END SOLUTION

res_q3b = pd.read_sql(query_q3b, engine)
res_q3b

,cand_name,cmte_nm
0,"ZUTLER, DANIEL PAUL MR",CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT
1,"ZUMWALT, JAMES",ZUMWALT FOR CONGRESS
2,"ZUKOWSKI, ANDREW GEORGE",ZUKOWSKI FOR CONGRESS
3,"ZUCCOLO, JOE",JOE ZUCCOLO FOR CONGRESS
4,"ZORN, ROBERT ERWIN",CONSTITUTIONAL COMMITTEE


In [23]:
# TEST
res_q3b.shape
# EXPECTED: (5, 2)

(5, 2)

In [24]:
# TEST
all(res_q3b == res_q3b.sort_values('cand_name', ascending=False))
# EXPECTED: True

True

In [25]:
# TEST
print(sorted(res_q3b['cmte_nm'].unique()))
# EXPECTED: ['CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT', 'CONSTITUTIONAL COMMITTEE', 'JOE ZUCCOLO FOR CONGRESS', 'ZUKOWSKI FOR CONGRESS', 'ZUMWALT FOR CONGRESS']

['CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT', 'CONSTITUTIONAL COMMITTEE', 'JOE ZUCCOLO FOR CONGRESS', 'ZUKOWSKI FOR CONGRESS', 'ZUMWALT FOR CONGRESS']


### Question 3c

As in the previous part, list the first 5 candidate names (`cand_name`) by reverse lexicographic order and their corresponding committee names (`cmte_nm`). This time, include all candidate names. If there is no matching `cand_id` in the `comm` table, then `cmte_nm` should be NULL.

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cand_name</th>
      <th>cmte_nm</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ZUTLER, DANIEL PAUL MR</td>
      <td>CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT</td>
    </tr>
    <tr>
      <th>1</th>
      <td>ZUMWALT, JAMES</td>
      <td>ZUMWALT FOR CONGRESS</td>
    </tr>
    <tr>
      <th>2</th>
      <td>ZUKOWSKI, ANDREW GEORGE</td>
      <td>ZUKOWSKI FOR CONGRESS</td>
    </tr>
    <tr>
      <th>3</th>
      <td>ZUCCOLO, JOE</td>
      <td>JOE ZUCCOLO FOR CONGRESS</td>
    </tr>
    <tr>
      <th>4</th>
      <td>ZORNOW, TODD MR</td>
      <td>None</td>
    </tr>
  </tbody>
</table>

```
BEGIN QUESTION
name: q3c
```

In [26]:
# BEGIN SOLUTION
query_q3c = '''
SELECT cand.cand_name, comm.cmte_nm
FROM cand LEFT JOIN comm ON cand.cand_id = comm.cand_id
ORDER BY cand_name DESC
LIMIT 5
'''
# END SOLUTION

res_q3c = pd.read_sql(query_q3c, engine)
res_q3c

,cand_name,cmte_nm
0,"ZUTLER, DANIEL PAUL MR",CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT
1,"ZUMWALT, JAMES",ZUMWALT FOR CONGRESS
2,"ZUKOWSKI, ANDREW GEORGE",ZUKOWSKI FOR CONGRESS
3,"ZUCCOLO, JOE",JOE ZUCCOLO FOR CONGRESS
4,"ZORNOW, TODD MR",None


In [27]:
# TEST
res_q3c.shape
# EXPECTED: (5, 2)

(5, 2)

In [28]:
# TEST
all(res_q3c == res_q3c.sort_values('cand_name', ascending=False))
# EXPECTED: True

True

In [29]:
# TEST
print(sorted(map(str, res_q3c['cmte_nm'].unique())))
# EXPECTED: ['CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT', 'JOE ZUCCOLO FOR CONGRESS', 'None', 'ZUKOWSKI FOR CONGRESS', 'ZUMWALT FOR CONGRESS']

['CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT', 'JOE ZUCCOLO FOR CONGRESS', 'None', 'ZUKOWSKI FOR CONGRESS', 'ZUMWALT FOR CONGRESS']
